## Notebook MySQL
En este notebook realizaremos la conexion e insercion de datos en mysql, ademas de las consultas e visualizacion de datos

In [ ]:
import mysql.connector
import time
import pandas as pd
import random
import logging
from faker import Faker

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# Funcion para medir el tiempo de ejecucion de una funcion
def medir_tiempo_ejecucion(func, *args, **kwargs):
    """Mide el tiempo de ejecucion de una funcion."""
    inicio = time.time()  # Guardar el tiempo de inicio
    resultado = func(*args, **kwargs)  # Ejecutar la funcion
    fin = time.time()  # Guardar el tiempo de fin
    tiempo_ejecucion = inicio - fin 
    print(f"Tiempo de ejecucion: {tiempo_ejecucion:.5f} segundos")
    return resultado

In [ ]:
# Inicializar Faker
fake = Faker()

# Funcion para generar el nombre
def generate_random_name():
    return fake.first_name()

# Funcion para generar el apellido
def generate_random_last_name():
    return fake.last_name()

# Funcion para generar el email
def generate_random_email():
    return fake.email()

# Funcion para generar el genero
def generate_random_gender():
    return random.choice(['Male', 'Female', 'Other'])

# Generar el millon de registros
data = {
    'id': [],
    'first_name': [],
    'last_name': [],
    'email': [],
    'gender': []
}

for i in range(22, 100001):
    data['id'].append(i)
    data['first_name'].append(generate_random_name())
    data['last_name'].append(generate_random_last_name())
    data['email'].append(generate_random_email())
    data['gender'].append(generate_random_gender())
    
    # Loguear cada 100,000 registros
    if i % 1000000 == 0:
        logging.info(f"Generados {i} registros")

# Crear DataFrame
df = pd.DataFrame(data)

# Guardar DataFrame en un archivo CSV
df.to_csv('/data/MOCK_DATA_LONG.csv', index=False)

logging.info("Generado correctamente archivo con 1 millon de registros")

In [ ]:
def wait_for_mysql(): # Intentar conectarse a MySQL
    while True:
        try:
            connection = mysql.connector.connect(
                host='mysql',
                user='root',
                password='rootpassword',
                connection_timeout=10
            )
            cursor = connection.cursor()
            print("Conexion exitosa a MySQL")
            return connection, cursor
        except Exception as e:
            print(f"Cargando MySQL... {e}")
            time.sleep(30)

In [ ]:
def insert_data_from_csv(connection, cursor, file_path): # Insertar los datos del CSV
    try:
        df = pd.read_csv(file_path)
        print("Archivo CSV leido correctamente")

        for index, row in df.iterrows():
            cursor.execute("""
                INSERT INTO users (id, first_name, last_name, email, gender)
                VALUES (%s, %s, %s, %s, %s)
            """, (row['id'], row['first_name'], row['last_name'], row['email'], row['gender']))
            print(f"Insertado registro: {row.to_dict()}")

        connection.commit()  # Asegurar que los datos se guarden
        print("Datos insertados correctamente")

    except Exception as e:
        print(f"Error al procesar el archivo CSV: {e}")

In [ ]:
def long_data_insert(connection, cursor):
    insert_data_from_csv(connection, cursor, '/data/MOCK_DATA_LONG.csv')
    rows = cursor.execute("SELECT * FROM user_data.users")
    df_rows = pd.DataFrame(list(rows))
    print(df_rows.head())


if __name__ == "__main__":
    connection, cursor = wait_for_mysql()
    try:
        medir_tiempo_ejecucion(long_data_insert, connection, cursor)
    finally:
        cursor.close()
        connection.close()

In [ ]:
def fetch_all_users(connection, cursor): # Devolver todos los usuarios
    try:
        cursor.execute("SELECT * FROM user_data.users")
        results = cursor.fetchall()
        
        print("Datos en la tabla 'users':")
        for row in results:
            print(row)
    except Exception as e:
        print(f"Error al recuperar datos: {e}")

if __name__ == "__main__":
    connection, cursor = wait_for_mysql()
    
    try:
        fetch_all_users(connection, cursor)  # Llamada a la funcion para comprobar los datos
    finally:
        cursor.close()
        connection.close()

In [ ]:
def get_user_by_id(connection, cursor, user_id): # Devolver un usuario por su ID
    try:
        result = cursor.execute("""
            SELECT * FROM user_data.users WHERE id = %s
        """, (user_id,))
        
        if result:
            for row in result:
                print(f"ID: {row.id}, Nombre: {row.first_name} {row.last_name}, Email: {row.email}, Genero: {row.gender}")
        else:
            print("Usuario no encontrado.")
    
    except Exception as e:
        print(f"Error al recuperar el usuario: {e}")

In [ ]:
def update_user_email(connection, cursor, user_id, new_email): # Actualizar el email de un usuario
    try:
        result = cursor.execute("""
            UPDATE user_data.users
            SET email = %s
            WHERE id = %s
        """, (new_email, user_id))
        print(f"Email actualizado para el usuario con ID {user_id}.")
    except Exception as e:
        print(f"Error al actualizar el email: {e}")

if __name__ == "__main__":
    connection, cursor = wait_for_mysql()
    
    try:
        medir_tiempo_ejecucion(update_user_email, connection, cursor, 1, "antonlago04@example.com")
        medir_tiempo_ejecucion(get_user_by_id, connection, cursor, 1)
    finally:
        cursor.close()
        connection.close()

In [ ]:
def get_emails(connection, cursor): # Devolver todos los emails
    try:
        result = cursor.execute("""
            SELECT email FROM user_data.users
        """)
        emails = [row.email for row in rows]
        for email in emails:
            print(email)
    except Exception as e:
        print(f"Error al ejecutar la funcion: {e}")
        return []

if __name__ == "__main__":
    connection, cursor = wait_for_mysql()
    
    try:
        medir_tiempo_ejecucion(get_emails, connection, cursor)
    finally:
        cursor.close()
        connection.close()